## Load library

In [1]:
import spacy # Analyse du language
import re
import en_core_web_sm
import en_core_web_lg
import math

## Find the characteres of GoT

In [3]:
nlp_sm = en_core_web_sm.load()
nlp_sm.max_length = 2500000

nlp_lg = en_core_web_lg.load() # cmd to download : python -m spacy download en_core_web_lg
nlp_lg.max_length = 2500000

#doc = nlp2(u"Apple is looking at buying U.K. startup for $1 billion") # u == unicode

#for ent in doc.ents:
#    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    
#doc = nlp1(u"Apple is looking at buying U.K. startup for $1 billion. Bran Stark is a boy.")
#for token in doc:
#    print(token.lemma_, token.text) 
#for ent in doc.ents:
#    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [3]:
# a = ['1','2','3','4']
# print(' '.join(a)) # Permet de concaténer les charactères d'une liste avec l'élément choisi
# name = word.replace('stuff','') # Remplace stuff par espace

In [4]:
def union_entities(listA, listB):
    result = listA
    for ent_b,y1,y2 in listB:
        overlap = False
        for ent_a,x1,x2 in listA:
            if x1 <= y2 and y1 <= x2: # overlap
                overlap = True
                if (y2-y1) > (x2-x2): # the largest entity is B
                    result.remove((ent_a,x1,x2))
                    result.append((ent_b,y1,y2))
                break
        if not overlap:
            result.append((ent_b,y1,y2))
    return result

In [5]:
def process_chapter(chapter):
    doc1 = nlp_sm(chapter)
    doc2 = nlp_lg(chapter)
    sm = []
    lg = []
    for ent in doc1.ents:
        if ent.label_ == 'PERSON':
            sm.append((ent.text, ent.start_char, ent.end_char))
    for ent in doc2.ents:
        if ent.label_ == 'PERSON':
            lg.append((ent.text, ent.start_char, ent.end_char))

    return union_entities(sm,lg)

In [8]:
def process_book(head, tail, book_number):
    filename = head + str(book_number) + tail
    with open(filename) as fp:
        book = fp.read()
    regex = r"\n{1,}([A-Z\s]+)\n{2,}"
    chunks = re.split(regex,book)
    
    entities = {}
    for index, chunk in enumerate(chunks):
        if chunk.isupper():
            #print(chunk.strip().title(),chunks[index+1][:100].strip())
            title = str(book_number) + '–' + str(math.floor(index/2)) + '-' + chunk.strip().title()
            chapter = chunks[index+1].strip().replace('\t','').replace('"','').replace('\n',' ').replace(' . . .','...')
            entities[title] = process_chapter(chapter)
    return entities

In [9]:
characters = {} #––––––––––––––––––––––––––––––––
for i in range (1,6):
    characters.update(process_book('GOT','.txt',i))
    print('book',i,'done')

book 1 done
book 2 done


KeyboardInterrupt: 

In [ ]:
with open ('GOT-characters-raw.pickle','wb+') as blav:
    pickle.dump(characters, blav)

In [4]:
#characters

In [5]:
#characters.keys()

In [6]:
#characters['1–1-Bran']